<a href="https://colab.research.google.com/github/aditya02shah/FaceRecognition/blob/main/FaceRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importing Libraries

In [1]:
#Standard Dependancies
from matplotlib import pyplot as plt
import cv2
import pandas as pd
import os
import random
import numpy as np
import tensorflow as tf

In [2]:
##Tensorflow dependencies-Functional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Layer,Conv2D,MaxPooling2D,Input,Flatten
import tensorflow as tf

In [3]:
POS_PATH=os.path.join('data','positive')
NEG_PATH=os.path.join('data','negative')
ANC_PATH=os.path.join('data','anchor')

In [ ]:
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

##Collecting Negatives

In [ ]:
!wget "http://vis-www.cs.umass.edu/lfw/lfw.tgz"

--2023-04-21 03:28:08--  http://vis-www.cs.umass.edu/lfw/lfw.tgz
Resolving vis-www.cs.umass.edu (vis-www.cs.umass.edu)... 128.119.244.95
Connecting to vis-www.cs.umass.edu (vis-www.cs.umass.edu)|128.119.244.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180566744 (172M) [application/x-gzip]
Saving to: ‘lfw.tgz’

lfw.tgz             100%[===================>] 172.20M  55.4MB/s    in 3.3s    

2023-04-21 03:28:12 (52.3 MB/s) - ‘lfw.tgz’ saved [180566744/180566744]



In [ ]:
##Uncompress Tar GZ Labelled Faces in the wild Dataset
!tar -xf lfw.tgz

In [ ]:
#Moving lfw images to data/negative
for directory in os.listdir('lfw'):
  for filename in os.listdir(os.path.join('lfw',directory)):
    CUR_PATH=os.path.join('lfw',directory,filename)
    NEW_PATH=os.path.join(NEG_PATH,filename)
    os.replace(CUR_PATH,NEW_PATH)

##Collect Positives and Anchors

In [ ]:
import uuid

In [ ]:
#Establish a connection to the webcam
cap=cv2.VideoCapture(3)
while cap.isOpened():
  ret,frame=cap.read()

  frame=frame[120:120+250,200:200+250,:]
  #Show image back to screen
  cv2.imshow("Image Collection",frame)

  #Collecting Anchors
  if cv2.waitKey(1) & 0XFF == ord('a'):
    imgname=os.path.join(ANC_PATH,'{}.jpg'.format(uuid.uuid1()))
    cv2.imwrite(imgname,frame)
  #Collecting Positives
  if cv2.waitKey(1) & 0XFF == ord('p'):
    imgname=os.path.join(POS_PATH,'{}.jpg'.format(uuid.uuid1()))
    cv2.imwrite(imgname,frame)

  #Breaking gracefull
  if cv2.waitKey(1) & 0XFF == ord('q'):
    break

#Release the webcam
cap.release()
#Close the image show frame
cv2.destroyAllWindows()

In [ ]:
cap.read??

In [4]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
!unzip '/content/gdrive/MyDrive/ML/data.zip'

##Pre-processing Data

In [6]:
anchor=tf.data.Dataset.list_files('/content/data/anchor/'+'*.jpg').take(300)
positive=tf.data.Dataset.list_files('/content/data/positive/'+'*.jpg').take(300)
negative=tf.data.Dataset.list_files('/content/data/negative/'+'*.jpg').take(300)

In [7]:
dir_test=anchor.as_numpy_iterator()
dir_test.next()

b'/content/data/anchor/8ea10020-deb2-11ed-a860-089798716103.jpg'

In [8]:
def preprocess(file_path):
  #Read in image from file path
  byte_img=tf.io.read_file(file_path)
  #Load in the image
  img=tf.io.decode_jpeg(byte_img)
  img=tf.image.resize(img,(100,100))
  img=img/255.0
  return img

##Create Labelled Dataset

In [9]:
positives=tf.data.Dataset.zip((anchor,positive,tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
##tensor_slices converts tf.ones(len(anchor)) into same datatype as anchor and positive(tf.data.Dataset files)
negatives=tf.data.Dataset.zip((anchor,negative,tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data=positives.concatenate(negatives)

In [10]:
data

<_ConcatenateDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None))>

In [11]:
samples=data.as_numpy_iterator()

In [12]:
example=samples.next()

In [13]:
example

(b'/content/data/anchor/8d9fdfb9-deb2-11ed-9dca-089798716103.jpg',
 b'/content/data/positive/fa648255-deb2-11ed-a51f-089798716103.jpg',
 1.0)

##Train and Test Partition

In [14]:
def preprocess_twin(input_img,validation_img,label):  
  return (preprocess(input_img),preprocess(validation_img),label)

In [15]:
# '*' is used to unpack the values
res=preprocess_twin(*example)

In [16]:
res[0]

<tf.Tensor: shape=(100, 100, 3), dtype=float32, numpy=
array([[[6.9436276e-01, 7.4926472e-01, 7.5318629e-01],
        [6.9828433e-01, 7.5318629e-01, 7.5710785e-01],
        [7.0122552e-01, 7.5612748e-01, 7.6004905e-01],
        ...,
        [6.8897057e-01, 7.5955880e-01, 8.0661762e-01],
        [6.9044119e-01, 7.5588238e-01, 8.0367649e-01],
        [6.8382353e-01, 7.5049019e-01, 7.8970587e-01]],

       [[6.9803923e-01, 7.5294119e-01, 7.5588238e-01],
        [6.9117647e-01, 7.4607843e-01, 7.4901962e-01],
        [6.9975489e-01, 7.5465685e-01, 7.5759804e-01],
        ...,
        [7.1004903e-01, 7.8455883e-01, 8.3946079e-01],
        [7.0563728e-01, 7.7720588e-01, 8.2622552e-01],
        [7.0661765e-01, 7.7720588e-01, 8.2426471e-01]],

       [[6.8308824e-01, 7.3799020e-01, 7.3799020e-01],
        [6.7745095e-01, 7.3235291e-01, 7.3235291e-01],
        [6.8823528e-01, 7.4313724e-01, 7.4313724e-01],
        ...,
        [6.9240195e-01, 7.6372552e-01, 8.0416667e-01],
        [7.0147061e-01

In [ ]:
plt.imshow(res[0])

##Data Pipeline

In [18]:
data=data.map(preprocess_twin)
data=data.cache()
data=data.shuffle(buffer_size=1024)

In [19]:

data

<_ShuffleDataset element_spec=(TensorSpec(shape=(100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None))>

In [20]:
ele=data.as_numpy_iterator()

In [21]:
sample=ele.next()

In [ ]:
plt.imshow(sample[1])

In [23]:
sample[2]

0.0

##Training and Test Partition

In [24]:
# Training partition
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [25]:
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

In [26]:
len(train_data),len(test_data)

(27, 12)

##Building Siamese Neural Network

####Embedding Layer

In [27]:
def make_embedding():
  inp=Input(shape=(100,100,3),name='input_image')

  #First Block
  c1=Conv2D(64,(10,10),activation='relu')(inp)
  m1=MaxPooling2D(64,(2,2),padding='same')(c1)

  #Second Block
  c2=Conv2D(128,(7,7),activation='relu')(m1)
  m2=MaxPooling2D(64,(2,2),padding='same')(c2)

  #Third Block
  c3=Conv2D(128,(4,4),activation='relu')(m2)
  m3=MaxPooling2D(64,(2,2),padding='same')(c3)

  #Final Embedding Block
  c4=Conv2D(256,(4,4),activation='relu')(m3)
  f1=Flatten()(c4)
  d1=Dense(4096,activation='sigmoid')(f1)

  return Model(inputs=[inp],outputs=[d1],name='embedding')

In [28]:
embedding=make_embedding()

In [29]:
embedding.summary()

Model: "embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_image (InputLayer)    [(None, 100, 100, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 91, 91, 64)        19264     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 46, 46, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 40, 40, 128)       401536    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 20, 20, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 17, 17, 128)       26

####Distance Layer

In [30]:
class L1Dist(Layer):
  def __init__(self,**kwargs):
    super().__init__()

  def call(self,input_embedding,validation_embedding):
    return tf.math.abs(input_embedding - validation_embedding)

In [31]:
L1Dist=L1Dist()

####Siamese Model

In [32]:
def make_siamese_model():
  #Handle input
  input_image=Input(name='input_img',shape=(100,100,3))
  validation_image=Input(name='validation_img',shape=(100,100,3))

  #Building siamese network components
  siamese_layer=L1Dist
  siamese_layer._name='distance'
  distances=siamese_layer(embedding(input_image),embedding(validation_image))

  #Classification Layer
  classifier=Dense(1,activation='sigmoid')(distances)

  return Model(inputs=[input_image,validation_image],outputs=classifier,name='SiameseNetwork')


In [33]:
siamese_model=make_siamese_model()

In [34]:
siamese_model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_img (InputLayer)         [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 validation_img (InputLayer)    [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 embedding (Functional)         (None, 4096)         38960448    ['input_img[0][0]',              
                                                                  'validation_img[0][

##Training

In [35]:
##Loss and Optimizer
binary_cross_loss=tf.losses.BinaryCrossentropy()
opt=tf.keras.optimizers.Adam(1e-4)

In [36]:
#Checkpoints
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

In [37]:
@tf.function
def train_step(batch):
  '''
  This function focuses on training a singlr batch
  '''
  with tf.GradientTape() as tape:
    #Get anchor and positive/negative images
    X=batch[:2]
    #Get label
    Y=batch[2]
    #Forward pass
    yhat=siamese_model(X,training=True)
    #Calculate loss
    loss=binary_cross_loss(Y,yhat)
  print(loss)
  #Calculate Gradients
  grad=tape.gradient(loss,siamese_model.trainable_variables)

  #Calculate updated weights and apply to siamese model
  opt.apply_gradients(zip(grad,siamese_model.trainable_variables))
  return loss

In [38]:
def train(data,EPOCHS):
  '''
  This function is used iterate over every batch in the dataset
  '''
  #Loop through epochs
  for epoch in range(1,EPOCHS+1):
    print("EPOCH {}/{}".format(epoch,EPOCHS))
    progbar=tf.keras.utils.Progbar(len(data))

    #Loop through each batch
    for idx,batch in enumerate(data):
      #Run train step here
      train_step(batch)
      progbar.update(idx+1)
    
    #Save checkpoints
    if epoch%10==0:
      checkpoint.save(file_prefix=checkpoint_prefix)

In [39]:
EPOCHS=50

In [40]:
train(train_data,EPOCHS)

EPOCH 1/50
Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
27/27 [==============================] - 18s 202ms/step
EPOCH 2/50
27/27 [==============================] - 5s 183ms/step
EPOCH 3/50
27/27 [==============================] - 5s 183ms/step
EPOCH 4/50
27/27 [==============================] - 5s 183ms/step
EPOCH 5/50
27/27 [==============================] - 5s 186ms/step
EPOCH 6/50
27/27 [==============================] - 5s 186ms/step
EPOCH 7/50
27/27 [==============================] - 5s 186ms/step
EPOCH 8/50
27/27 [==============================] - 5s 185ms/step
EPOCH 9/50
27/27 [==============================] - 5s 186ms/step
EPOCH 10/50
27/27 [==============================] - 5s 188ms/step
EPOCH 11/50
27/27 [==============================] - 5s 193ms/step
EPOCH 12/50
27/27 [==============================] - 5s 189ms/step
EPOCH 13/50
27/27 [==============================]

##Evaluate Model

In [41]:
from tensorflow.keras.metrics import Precision,Recall

In [42]:
#Getting a batch of test data
test_input,test_val,y_true=test_data.as_numpy_iterator().next()

In [43]:
#Making Predictions
yhat=siamese_model.predict([test_input,test_val])

1/1 [==============================] - 0s 168ms/step


In [44]:
yhat

array([[1.0000000e+00],
       [1.4850872e-15],
       [9.9999142e-01],
       [1.0000000e+00],
       [6.4135183e-11],
       [1.0000000e+00],
       [1.0000000e+00],
       [9.9999964e-01],
       [8.7996703e-09],
       [9.9999464e-01],
       [1.3727394e-12],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.4531632e-12],
       [7.9203331e-11],
       [2.6994746e-06]], dtype=float32)

In [45]:
#Post-Processing the results
[1 if prediction>0.5 else 0 for prediction in yhat]

[1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0]

In [46]:
y_true

array([1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0.],
      dtype=float32)

In [47]:
#Recall-Measures how many actual positives are correctly predicted by model

#Creating a Metric Object
m=Recall()
#Calculating Recall Value
m.update_state(y_true,yhat)
#Return Recall Result
m.result().numpy()

1.0

In [48]:
#Precision-Measures how many of the predicted positive instances are actually positive

p=Precision()
p.update_state(y_true,yhat)
p.result().numpy()

1.0

####Visualise Results

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
plt.imshow(test_input[0])
plt.subplot(1,2,2)
plt.imshow(test_val[0])

##Save Model

In [50]:
siamese_model.save('siamese_model.h5')

In [51]:
#Loading Model
siamese_model = tf.keras.models.load_model('/content/siamese_model.h5', 
                                   custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

##Real Time Test

In [ ]:
def verify(model,detection_threshold,verification_threshold):
  '''
  Detection Threhold-Metric above which image is considered positive
  Verification Threhold-Proportion of positive predictions/total positive samples
  '''
  results=[]
  for image in os.listdir(os.path.join('application_data','verification_images')):
    input_img=preprocess(os.path.join('application_data','input_images','input_image.jpg'))
    validation_img=preprocess(os.path.join('application_data','verification_images',image))

    result=model.predict(list(np.expand_dims([input_img,verification_threshold],axis=1)))
    results.append(result)

  detection=np.sum(np.array(results)>detection_threshold)
  verification=detection/len(os.listdir(os.path.join('application_data','verification_images')))
  verified=verification>verification_threshold
  return verified

In [ ]:
cap = cv2.VideoCapture(4)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[120:120+250,200:200+250, :]
    
    cv2.imshow('Verification', frame)
    
    # Verification trigger
    if cv2.waitKey(10) & 0xFF == ord('v'):
        cv2.imwrite(os.path.join('application_data', 'input_image', 'input_image.jpg'), frame)
        # Run verification
        results, verified = verify(siamese_model, 0.9, 0.7)
        print(verified)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()